<a href="https://colab.research.google.com/github/hybrits/H-ALLY/blob/main/Primer_entramientoV_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast

# Cargar el archivo CSV
df = pd.read_csv('/content/drive/MyDrive/dataset_limpio1.csv')

# Crear un mapeo de etiquetas único para cada valor en la columna 'Título'
label_mapping = {label: idx for idx, label in enumerate(df['Título'].unique())}

# Aplicar el mapeo de etiquetas a la columna 'Título'
df['Título'] = df['Título'].map(label_mapping)

# Verificar los valores NaN en 'Título'
nan_titles = df['Título'].isna()
print("Filas con NaN en 'Título':")
print(df[nan_titles])

# Manejar valores NaN: Opción 1 - Eliminar filas con NaN
df = df.dropna(subset=['Título'])

# Convertir 'Título' a enteros después de manejar NaN
df['Título'] = df['Título'].astype(int)

# Verificar que no queden NaN en 'Título'
print("Número de valores NaN en 'Título' después de la limpieza:", df['Título'].isna().sum())

# Asegurarse de que todos los valores en 'Contenido' sean cadenas de texto
df['Contenido'] = df['Contenido'].fillna('').astype(str)

# Tokenización usando DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(df['Contenido'].tolist(), truncation=True, padding=True, max_length=512)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(train_encodings['input_ids'], df['Título'], test_size=0.2, random_state=42)

print("Datos preparados para el entrenamiento.")


Filas con NaN en 'Título':
Empty DataFrame
Columns: [Título, URL, Autor, Fecha, Contenido]
Index: []
Número de valores NaN en 'Título' después de la limpieza: 0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Datos preparados para el entrenamiento.


In [3]:
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Verificar si hay una GPU disponible y configurarla para el entrenamiento
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Crear el modelo
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_mapping))
model.to(device)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida para guardar los resultados
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=8,   # Tamaño del lote por dispositivo durante el entrenamiento
    per_device_eval_batch_size=8,    # Tamaño del lote por dispositivo durante la evaluación
    warmup_steps=500,                # Número de pasos para el calentamiento
    weight_decay=0.01,               # Tasa de decaimiento del peso
    logging_dir='./logs',            # Directorio de registro para TensorBoard
    logging_steps=10,
    eval_strategy="epoch"            # Evaluación al final de cada época
)

# Convertir las entradas y etiquetas de entrenamiento y prueba a tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.long).detach().clone()
X_test = torch.tensor(X_test, dtype=torch.long).detach().clone()
y_train = torch.tensor(y_train.values, dtype=torch.long).detach().clone()
y_test = torch.tensor(y_test.values, dtype=torch.long).detach().clone()

# Crear un dataset a partir de las entradas y etiquetas
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Crear datasets de entrenamiento y prueba
train_dataset = CustomDataset(
    {
        'input_ids': X_train,
        'attention_mask': train_encodings['attention_mask'][:len(X_train)]
    },
    y_train
)

test_dataset = CustomDataset(
    {
        'input_ids': X_test,
        'attention_mask': train_encodings['attention_mask'][len(X_train):]
    },
    y_test
)

# Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
trainer.evaluate()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Guardar el modelo y el tokenizador
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [33]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

# Cargar el modelo y el tokenizador guardados
model = DistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/trained_model')
tokenizer = DistilBertTokenizerFast.from_pretrained('/content/drive/MyDrive/trained_model')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [39]:
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Configurar el dispositivo (CPU o GPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Cargar el archivo CSV
df = pd.read_csv('/content/drive/MyDrive/dataset_limpio1.csv')

# Crear un mapeo de etiquetas único para cada valor en la columna 'Título'
label_mapping = {label: idx for idx, label in enumerate(df['Título'].unique())}

# Aplicar el mapeo de etiquetas a la columna 'Título'
df['Título'] = df['Título'].map(label_mapping)

# Manejar valores NaN: Eliminar filas con NaN
df = df.dropna(subset=['Título'])

# Convertir 'Título' a enteros después de manejar NaN
df['Título'] = df['Título'].astype(int)

# Asegurarse de que todos los valores en 'Contenido' sean cadenas de texto
df['Contenido'] = df['Contenido'].fillna('').astype(str)

# Crear un diccionario de soluciones basado en el dataset
solutions_db = {}
for idx, row in df.iterrows():
    label = list(label_mapping.keys())[list(label_mapping.values()).index(row['Título'])]
    if label not in solutions_db:
        solutions_db[label] = []
    solutions_db[label].append(row['Contenido'])

# Cargar el modelo y el tokenizador guardados
model = DistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/trained_model')
tokenizer = DistilBertTokenizerFast.from_pretrained('/content/drive/MyDrive/trained_model')
model.to(device)

# Consulta de ejemplo
new_problem = ["Como hacer un overclocking"]

# Tokenizar la consulta
inputs = tokenizer(new_problem, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Mover los tensores a la GPU si está disponible
inputs = {key: val.to(device) for key, val in inputs.items()}

# Poner el modelo en modo de evaluación
model.eval()

# Desactivar el cálculo de gradientes para la predicción
with torch.no_grad():
    outputs = model(**inputs)

# Obtener las predicciones
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

# Convertir las predicciones a etiquetas legibles
predicted_label = list(label_mapping.keys())[list(label_mapping.values()).index(predictions.item())]

print(f"La categoría del problema es: {predicted_label}")

# Preparar la búsqueda de soluciones con TF-IDF
df['combined_text'] = df['Título'].astype(str) + " " + df['Contenido']

# Crear una representación TF-IDF de los textos
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])

# Transformar la consulta en una representación TF-IDF
query_tfidf = vectorizer.transform([new_problem[0]])

# Calcular las similitudes de coseno entre la consulta y el conjunto de datos
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

# Filtrar los índices que corresponden a la categoría predicha
filtered_indices = [i for i in range(len(df)) if df.iloc[i]['Título'] == predictions.item()]

# Encontrar el índice del texto más similar dentro de la categoría predicha
most_similar_index = max(filtered_indices, key=lambda x: cosine_similarities[x])

# Obtener la solución más similar
suggested_solution = df.iloc[most_similar_index]['Contenido']

print(f"Sugerencia de solución: {suggested_solution}")


La categoría del problema es: [SOLVED] First successful Overclock
Sugerencia de solución: Re: First successful Overclock

overclocking a card or cpu can help in games but really these days with the CPUs and GPUs there really isn't a need.

I always overclock because I always have but I generally buy factory overclocked cards and just focus on the cpu.
